In [6]:
import pandas as pd

df = pd.read_csv("data_mutations2.tsv", sep="\t")
dx = pd.read_csv("paired_bladder_2022_clinical_data.tsv", sep="\t")
df2 = df[['Hugo_Symbol', 'Variant_Classification', 'Tumor_Sample_Barcode']]
df3 = dx[['Sample ID', 'Intravesical Treatment', 'Overall Survival Status', 'Sample Type']]
df2 = df2.rename(columns={"Tumor_Sample_Barcode": "Sample ID"})
df4 = df3.merge(df2, on=['Sample ID'], how='inner')
print(df4.head())
df4.to_csv("data_mutations4.csv", sep=",", index=False)

C:\Users\pc\AppData\Local\Temp\ipykernel_11204\1859903537.py:3: DtypeWarning: Columns (17,18,19,32,47,53,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_mutations2.tsv", sep="\t")


           Sample ID Intravesical Treatment Overall Survival Status   
0  P-0000034-T01-IM3                      0              1:DECEASED  \
1  P-0000034-T01-IM3                      0              1:DECEASED   
2  P-0000034-T01-IM3                      0              1:DECEASED   
3  P-0000034-T01-IM3                      0              1:DECEASED   
4  P-0000034-T01-IM3                      0              1:DECEASED   

  Sample Type Hugo_Symbol Variant_Classification  
0     Primary      ARID1A      Nonsense_Mutation  
1     Primary        RYBP      Missense_Mutation  
2     Primary        FAT1      Nonsense_Mutation  
3     Primary       FGFR4      Missense_Mutation  
4     Primary         BLM      Nonsense_Mutation  


In [5]:
import pandas as pd

df = pd.read_csv("data_mutations4.tsv", sep="\t")
df['Intravesical Treatment'] = df['Intravesical Treatment'].dropna()
df = df[df['Intravesical Treatment'].isin(['0','1','2','3'])]


# crear las clases tratamiento x alive
clases = {
    1:[0,1],
    2:[0,1],
    3:[0,1],
    4:[0,1],
    5:[0,1],
    6:[0,1],
    7:[0,1],
    8:[0,1],
    9:[0,1],
    10:[0,1],
}
mascara = df['Sample Type'].str.contains('unknown', case=False)
df = df[~mascara]
def cambiar_tratamiento(value):
        dicc={
            '0':'N',
            '1':'BCG',
            '2':'CH',
            '3':'BO',
        }
        return dicc[value]
def cambiar(value):
        dicc={
            'Nonsense_Mutation': 'NS',
            'Missense_Mutation': 'MS',
            'Frame_Shift_Del': 'FS',
            "5'Flank": '5F',
            'In_Frame_Ins': 'IFI',
            'In_Frame_Del': 'IFD',
            'Splice_Site': 'SS',
            'Frame_Shift_Ins': 'FSI',
            'Translation_Start_Site': 'TSS',
            'Nonstop_Mutation': 'NM',
            'Intron': 'I',
            'Splice_Region': 'SR',
            "3'Flank": '3F',
            "5'UTR": '5U',
            'Silent': 'S',
        }
        return dicc[value]
df['Variant_Classification'] = df['Variant_Classification'].apply(lambda x: cambiar(x) )
df['Intravesical Treatment'] = df['Intravesical Treatment'].apply(lambda x: cambiar_tratamiento(x) )
print(df['Intravesical Treatment'].unique())

df['Overall Survival Status'] = df['Overall Survival Status'].apply(lambda x: 'D' if x == '1:DECEASED' else 'A')

df['Sample Type'] = df['Sample Type'].replace({'Primary': 'P', 'Metastasis': 'M', 'Local Recurrence': 'L'})
df = df.rename(columns={'Variant_Classification': 'Mutation_Type', 'Intravesical Treatment':
    'Treatment', 'Overall Survival Status': 'Result','Hugo_Symbol': 'Gene', 'Sample Type': 'Sample_Type', 'Sample ID': 'Sample_ID'})
# por cada registro crear una columna combinando todos los valores de tipo de muestra, tipo mutacion y gen

df.to_csv("data_mutations5.csv", sep=",", index=False)


['N' 'BO' 'BCG' 'CH']


In [6]:
import pandas as pd

# Cargamos el archivo CSV original
df = pd.read_csv('data_mutations5.csv')
import pandas as pd
from itertools import product

# Supongamos que tienes un DataFrame llamado df
# con tres columnas: 'col1', 'col2' y 'col3'

# Obtenemos los valores únicos de cada columna
unique_values_col1 = df['Sample_Type'].unique()
unique_values_col2 = df['Mutation_Type'].unique()
unique_values_col3 = df['Gene'].unique()

# Generamos todas las combinaciones posibles de valores únicos
combinations = list(product(unique_values_col3, unique_values_col1, unique_values_col2))
# Crear un DataFrame vacío con columnas
new_df = pd.DataFrame(columns=["-".join(map(str, combination)) for combination in combinations])

new_df.to_csv("data_mutations6.csv", sep=",", index=False)

In [7]:
import pandas as pd

# Cargar los datos desde el CSV en un DataFrame
df = pd.read_csv("data_mutations5.csv")

# Crear un nuevo DataFrame resultado_df con las mismas columnas que df
resultado_df = df.copy()
print(resultado_df)
for paciente_id in df["Sample_ID"].str.extract(r'P-(\d+)')[0].unique():
    # Filtrar las filas para el paciente actual
    paciente_rows = df[df["Sample_ID"].str.contains(f"P-{paciente_id}")]

    # Iterar a través de las filas del paciente
    for _, row in paciente_rows.iterrows():
        # Obtener los valores de las columnas 3, 4 y 5
        valores = row[["Sample_Type","Gene","Mutation_Type"]].values

        # Marcar las filas correspondientes en resultado_df con 1
        resultado_df.loc[paciente_rows.index, valores] = 1
    print(resultado_df)
'''
# Iterar a través de cada paciente
for paciente_id in df["Registro"].str.extract(r'P-(\d+)')[0].unique():
    # Filtrar las filas para el paciente actual
    paciente_rows = df[df["Registro"].str.contains(f"P-{paciente_id}")]

    # Iterar a través de las filas del paciente
    for _, row in paciente_rows.iterrows():
        # Obtener los valores de las columnas 3, 4 y 5
        valores = row[["Columna3", "Columna4", "Columna5"]].values

        # Marcar las filas correspondientes en resultado_df con 1
        resultado_df.loc[paciente_rows.index, valores] = 1

# Rellenar las celdas con NaN (no marcadas) con 0
resultado_df = resultado_df.fillna(0)

# Restablecer el índice del nuevo DataFrame
resultado_df = resultado_df.reset_index(drop=True)

# Ahora resultado_df contiene las marcas 1 y 0 según la lógica que especificaste
print(resultado_df)
'''



               Sample_ID Treatment Result Sample_Type     Gene Mutation_Type
0      P-0000034-T01-IM3         N      D           P   ARID1A            NS
1      P-0000034-T01-IM3         N      D           P     RYBP            MS
2      P-0000034-T01-IM3         N      D           P     FAT1            NS
3      P-0000034-T01-IM3         N      D           P    FGFR4            MS
4      P-0000034-T01-IM3         N      D           P      BLM            NS
...                  ...       ...    ...         ...      ...           ...
21853  P-0055829-T01-IM6       BCG      A           P  ANKRD11            NS
21854  P-0055829-T01-IM6       BCG      A           P     BRD4            MS
21855  P-0055829-T01-IM6       BCG      A           P    BARD1            MS
21856  P-0055829-T01-IM6       BCG      A           P      VHL            MS
21857  P-0055829-T01-IM6       BCG      A           P    KDM6A           FSI

[21858 rows x 6 columns]
               Sample_ID Treatment Result Sample_T

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Cargamos el archivo CSV original
df = pd.read_csv('data_mutations5.csv')
import pandas as pd
from itertools import product

# Supongamos que tienes un DataFrame llamado df
# con tres columnas: 'col1', 'col2' y 'col3'

unique_values_col3 = df['Gene'].unique()

# Generamos todas las combinaciones posibles de valores únicos
#combinations = list(product(unique_values_col3, unique_values_col2))
combinations = list(product(unique_values_col3))
# Crear un DataFrame vacío con columnas
#columns =[combination[0]+'-'+combination[1] for combination in combinations]
columns =[combination[0] for combination in combinations]
columns.append('Class')
new_df = pd.DataFrame(columns=columns)

new_df.to_csv("data_mutations6aux2.csv", sep=",", index=False)

In [1]:
import pandas as pd

# Supongamos que tienes tu dataframe original llamado df_original
df_original = pd.read_csv('data_mutations5.csv')
# Luego, tienes un dataframe con las columnas de genes
genes_df = pd.read_csv('data_mutations6aux2.csv')

# Obtén una lista de pacientes únicos
pacientes_unicos = df_original['Sample_ID'].unique()

# Itera sobre cada paciente único y llena el dataframe de genes
for paciente in pacientes_unicos:
    # Filtra las filas correspondientes a este paciente
    genes_paciente = df_original[df_original['Sample_ID'] == paciente]['Gene'].tolist()
    gen_tipo_muestra = [f"{gen}" for gen in genes_paciente]

    # Crea una nueva fila en el dataframe de genes con los valores adecuados
    nueva_fila = {gen: 1 if gen in gen_tipo_muestra else 0 for gen in genes_df.columns}

    # Agrega la clase
    tratamiento_resultado = df_original[df_original['Sample_ID'] == paciente][['Treatment', 'Result']].iloc[0]
    nueva_fila['Class'] = f"{tratamiento_resultado['Treatment']}_{tratamiento_resultado['Result']}"

    # Convierte la nueva fila en un DataFrame temporal
    nueva_fila_df = pd.DataFrame([nueva_fila], columns=genes_df.columns)

    # Concatena el DataFrame temporal al DataFrame de genes
    genes_df = pd.concat([genes_df, nueva_fila_df], ignore_index=True)

print(genes_df)
genes_df.to_csv("data_mutations8.csv", sep=",", index=False)

     ARID1A RYBP FAT1 FGFR4 BLM CIC KDM6A PIK3CA NOTCH2 DDR2  ... PDPK1 RRAS2   
0         1    1    1     1   1   1     1      0      0    0  ...     0     0  \
1         0    0    1     0   0   0     0      1      1    1  ...     0     0   
2         0    0    0     1   0   0     0      0      0    0  ...     0     0   
3         0    0    1     0   0   0     0      0      0    0  ...     0     0   
4         0    0    0     0   0   0     0      0      0    0  ...     0     0   
...     ...  ...  ...   ...  ..  ..   ...    ...    ...  ...  ...   ...   ...   
1243      0    0    0     0   0   0     0      0      0    0  ...     0     0   
1244      0    0    0     0   0   0     0      0      0    0  ...     0     0   
1245      0    0    0     0   0   0     0      0      0    0  ...     0     0   
1246      0    0    0     0   1   0     1      0      0    0  ...     0     0   
1247      0    0    0     0   0   0     1      0      0    0  ...     0     0   

     MDC1-AS1 MAPKAP1 PTP4A

In [2]:
import pandas as pd

# Supongamos que tienes tu dataframe original llamado df_original
df_original = pd.read_csv('data_mutations5.csv')
# Luego, tienes un dataframe con las columnas de genes
genes_df = pd.read_csv('data_mutations6aux.csv')

# Obtén una lista de pacientes únicos
pacientes_unicos = df_original['Sample_ID'].unique()

# Itera sobre cada paciente único y llena el dataframe de genes
for paciente in pacientes_unicos:
    # Filtra las filas correspondientes a este paciente
    genes_paciente = df_original[df_original['Sample_ID'] == paciente]['Gene'].tolist()
    tipo_muestra_paciente = df_original[df_original['Sample_ID'] == paciente]['Sample_Type'].tolist()
    gen_tipo_muestra = [f"{gen}-{tipo_muestra}" for gen, tipo_muestra in zip(genes_paciente, tipo_muestra_paciente)]

    # Crea una nueva fila en el dataframe de genes con los valores adecuados
    nueva_fila = {gen: 1 if gen in gen_tipo_muestra else 0 for gen in genes_df.columns}

    # Agrega la clase
    tratamiento_resultado = df_original[df_original['Sample_ID'] == paciente][['Treatment', 'Result']].iloc[0]
    nueva_fila['Class'] = f"{tratamiento_resultado['Treatment']}_{tratamiento_resultado['Result']}"

    # Convierte la nueva fila en un DataFrame temporal
    nueva_fila_df = pd.DataFrame([nueva_fila], columns=genes_df.columns)

    # Concatena el DataFrame temporal al DataFrame de genes
    genes_df = pd.concat([genes_df, nueva_fila_df], ignore_index=True)

print(genes_df)
genes_df.to_csv("data_mutations8.csv", sep=",", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data_mutations5.csv'

In [12]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import pandas as pd

from sklearn.decomposition import PCA
df = pd.read_csv('data_mutations8.csv')
# Normalizamos los datos para que tengan media 0 y varianza 1
scaler = StandardScaler()
X = scaler.fit_transform(df[df.columns[:-1]])
X

array([[ 1.58291439,  9.06642157,  2.67523134, ...,  0.        ,
         0.        ,  0.        ],
       [-0.6317461 , -0.1102971 ,  2.67523134, ...,  0.        ,
         0.        ,  0.        ],
       [-0.6317461 , -0.1102971 , -0.37379945, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.6317461 , -0.1102971 , -0.37379945, ...,  0.        ,
         0.        ,  0.        ],
       [-0.6317461 , -0.1102971 , -0.37379945, ...,  0.        ,
         0.        ,  0.        ],
       [-0.6317461 , -0.1102971 , -0.37379945, ...,  0.        ,
         0.        ,  0.        ]])

In [6]:
# Ejecutamos PCA con 5 componentes principales
pca = PCA(n_components=100)
pca.fit(X)
pca

PCA(n_components=100)

In [9]:
import numpy as np

# Seleccionamos los componentes principales con varianza más alta
explained_variance_ratio = pca.explained_variance_ratio_
important_components = np.argsort(explained_variance_ratio)[-10:]
important_components

array([19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
        2,  1,  0], dtype=int64)

In [10]:
# Identificamos los genes asociados con los componentes principales seleccionados
important_genes = df.columns[:-1][important_components]
important_genes

Index(['IL7R', 'TP53', 'FANCA', 'IGF1R', 'IRS2', 'CDKN2A', 'KMT2C', 'EPHB1',
       'FGFR3', 'CDKN1A', 'NOTCH4', 'BAP1', 'TERT', 'KDM6A', 'CIC', 'BLM',
       'FGFR4', 'FAT1', 'RYBP', 'ARID1A'],
      dtype='object')

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv("data_mutations7.csv")
target=df["Class"]
# Seleccionamos los genes más relevantes
important_genes = df.columns[:-1][np.argsort(pca.explained_variance_ratio_)[-5:]]

# Recortamos el conjunto de datos a los genes más relevantes
df = df[important_genes]
# Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(df[important_genes], target, test_size=0.25)

# Entrenamos el modelo
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluamos el rendimiento del modelo
precision = model.score(X_test, y_test)
print("Precisión:", precision)

Precisión: 0.3556701030927835


In [51]:
import pandas as pd

# Cargamos el archivo CSV original
df = pd.read_csv('data_mutations7.csv')
df=df[list(
    {'AKT1', 'TERT', 'DUSP4', 'FGFR3', 'TCF3', 'H3C11', 'SPEN', 'TERT', 'TP53', 'KMT2D', 'KMT2D', 'SMYD3', 'BMPR1A',
     'ERCC2', 'FLT4', 'KDM6A', 'SOX17', 'ARID1A', 'H3F3B', 'NKX2-1'})+['Class']]
df.to_csv("data_mutations_f1.csv", sep=",", index=False)


In [3]:
import pandas as pd

df = pd.read_csv("data_mutations8.csv")
df = df[df['Class'].isin(['N_D','N_A','BO_D','BO_A','BCG_D','BCG_A'])]
df.to_csv("data_mutations_balance.csv", sep=",", index=False)


In [1]:
#para sacar por frecuencias

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv("data_mutations8.csv")
binary_columns = df.drop(columns=['Class'])
value_counts = binary_columns.apply(pd.Series.value_counts)
sorted_columns = value_counts.sort_values(by=1, axis=1, ascending=False)
# Obtener las columnas ordenadas
ordered_columns = sorted_columns.columns

# Crear un nuevo DataFrame con las columnas ordenadas
df_ordered['Class'] = df['Class']
counts_row = binary_columns.sum()
df_ordered.loc[len(df)] = counts_row

NameError: name 'df_ordered' is not defined

In [9]:
df_ordered.to_csv("data_mutations9.csv", sep=",", index=False)